In [25]:
from seleniumbase import Driver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException

In [26]:
driver = Driver(uc=True)   # undetected-chromedriver

In [27]:
BASE_URL = 'https://www.amazon.eg/s?k=%D8%A7%D9%84%D8%A7%D8%AF%D9%88%D8%A7%D8%AA+%D8%A7%D9%84%D8%B5%D8%AD%D9%8A%D9%87+%D9%84%D9%84%D8%AD%D9%85%D8%A7%D9%85+%D8%AF%D8%B4&language=ar_AE&crid=25LOXKZSXIVJV&sprefix=%2Caps%2C119&ref=nb_sb_ss_recent_1_0_recent'

In [28]:
all_urls = []
driver.get(BASE_URL)
time.sleep(5)  # allow dynamic content to load


In [29]:
for i in range(1,8):
    links = driver.find_elements(By.CSS_SELECTOR, "a.a-link-normal.s-no-outline")
    for link in links:
        url = link.get_attribute("href")
        if url and "/dp/" in url:  # filter real product links
            all_urls.append(url)
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.s-pagination-next")
        if "s-pagination-disabled" in next_button.get_attribute("class"):
            break  # No more pages
        next_button.click()
        time.sleep(5)  # wait for next page to load
    except NoSuchElementException:
        break  # No "Next" button found, exit loop


In [30]:
all_urls

['https://www.amazon.eg/%D9%85%D8%AC%D9%85%D9%88%D8%B9%D8%A9-%D8%AE%D9%84%D8%A7%D8%B7%D8%A7%D8%AA-%D9%85%D8%B7%D8%A8%D8%AE-%D9%88%D8%AD%D9%85%D8%A7%D9%85-%D8%A7%D9%84%D9%83%D8%B1%D9%88%D9%85/dp/B0FDF8FF8X/ref=sr_1_1?crid=25LOXKZSXIVJV&dib=eyJ2IjoiMSJ9.DVzHgJ__hU86ua9e9FXJ86I72kdE17OMPi36zFmPdvAfsIheROrmb_8pzy0d-P-2Iuy0xObQusR0_gqpwk2da3ZhIdFRQPLEQh3DOcI_ENaGo7E4U6GDqr3BGHNGCqgo4-r-QydG3N1RsVYBD4qNaQFtK5RFkEM0oRkeZV47Lr-OoqyRu98U49mFyrZuWWN81xtkrwzuTkXJlOPodI9AIx_F2LeEGMGoT5FmaU-H5412h4pmAv6RHDbUfWAyneVKg0tH_KGN_EtxjQrxWOGDStvnGnxrvrpu5jNYVmt2fRE.MsjPOxDqYrKZZylJbBFkmjgayqAjnjsDKygSosuO-Lw&dib_tag=se&keywords=%D8%A7%D9%84%D8%A7%D8%AF%D9%88%D8%A7%D8%AA+%D8%A7%D9%84%D8%B5%D8%AD%D9%8A%D9%87+%D9%84%D9%84%D8%AD%D9%85%D8%A7%D9%85+%D8%AF%D8%B4&qid=1766003456&sprefix=%2Caps%2C119&sr=8-1',
 'https://www.amazon.eg/%D8%A5%D8%B3%D8%AA%D8%A7%D9%86%D9%84%D8%B3-%D8%A7%D9%84%D8%AD%D9%85%D8%A7%D9%85-%D8%A7%D9%84%D8%B4%D8%A7%D9%88%D8%B1-%D8%A7%D9%84%D9%85%D9%8A%D8%A7%D9%87-%D8%AD%D9%85%D8%A7%D9%85%D9%83/

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def extract_amazon_reviews(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/124.0.0.0 Safari/537.36"
        ),
        "Accept-Language": "en-US,en;q=0.9"
    }

    html = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html, "html.parser")

    # Product name
    product_tag = soup.find("span", {"id": "productTitle"})
    product_name = product_tag.get_text(strip=True) if product_tag else "Unknown Product"

    reviews = []

    review_blocks = soup.find_all("li", {"data-hook": "review"})

    for r in review_blocks:
        review_id = r.get("id")

        # Rating: numeric only
        rating_tag = r.find("i", {"data-hook": "review-star-rating"})
        if rating_tag:
            rating_text = rating_tag.text.strip()  # e.g., "5.0 out of 5 stars"
            rating_match = re.search(r"[\d.]+", rating_text)
            rating = float(rating_match.group()) if rating_match else None
        else:
            rating = None

        # Title: remove leading rating if present
        title_tag = r.find("a", {"data-hook": "review-title"})
        if title_tag:
            title_text = title_tag.get_text(strip=True)
            title = re.sub(r"^\d+(\.\d+)? out of 5 stars", "", title_text).strip()
        else:
            title = None

        # Date: remove "Reviewed in <Country> on"
        date_tag = r.find("span", {"data-hook": "review-date"})
        if date_tag:
            date_text = date_tag.text.strip()  # e.g., "Reviewed in Egypt on 26 July 2025"
            date_match = re.search(r"on (.+)$", date_text)
            date = date_match.group(1).strip() if date_match else date_text
        else:
            date = None

        # Body
        body_tag = r.find("span", {"data-hook": "review-body"})
        if body_tag:
            original = body_tag.find("span", class_="cr-original-review-content")
            body = original.text.strip() if original else body_tag.text.strip()
        else:
            body = None

        reviews.append({
            "product": product_name,
            "review_id": review_id,
            "rating": rating,
            "title": title,
            "date": date,
            "text": body
        })

    return reviews


In [32]:
all_reviews = []
for url in all_urls:
    reviews = extract_amazon_reviews(url)
    all_reviews.extend(reviews)  # Combine all reviews in one list

# Convert to DataFrame
df_all_reviews = pd.DataFrame(all_reviews)


In [33]:
df_all_reviews.head()

,product,review_id,rating,title,date,text
0,Chrome Kitchen and Bathroom Mixer Set - Includ...,R3RROEQVY41FJK,2.0,مش بتاع استخدام كتير عايز استخدام خفيف,30 November 2025,بالنسبة للسعر معقول بس جالي المقبض بتاع الشاور...
1,Chrome Kitchen and Bathroom Mixer Set - Includ...,R1OZNWYVX8RDGX,1.0,سيء,24 July 2025,منتج سيء بكل ما تحتويه الكلمة من معنيالخلاط نف...
2,Chrome Kitchen and Bathroom Mixer Set - Includ...,RGDAHIJ96UFS2,5.0,ممتازة رائع خامة معقولة بالنسبة للسعر,26 July 2025,خلاط رائع تقيل
3,مقلاة دش استانلس مربع تيلة أو رأس دش الحمام ال...,ROAOX94JBQ96K,5.0,ممتازة وتقيلة ومحملة والفينش بتاعها هاااايل حل...,27 August 2025,هايلة
4,مقلاة دش استانلس مربع تيلة أو رأس دش الحمام ال...,R1GMHNAKIQT21I,5.0,ممتاز,26 August 2024,خامه استانلس وقويه جدا


In [34]:
df_all_reviews.duplicated().sum()

np.int64(77)

In [35]:
df_all_reviews.drop_duplicates(inplace=True)

In [36]:
df_all_reviews.shape

(350, 6)

In [37]:
df_all_reviews.to_csv("all_amazon_reviews.csv", index=False)